In [ ]:
from tensorflow.keras.saving import register_keras_serializable
from tensorflow.keras import backend as K  # Assuming you're using Keras backend
import cv2
import numpy as np
import matplotlib.pyplot as plt

@register_keras_serializable()
def get_weighted_loss(pos_weights, neg_weights, epsilon=1e-7):
  """
  Defines a custom weighted loss function.

  Args:
      pos_weights: List of weights for positive classes.
      neg_weights: List of weights for negative classes.
      epsilon: Small value to avoid log(0) errors.

  Returns:
      A function that calculates the weighted loss.
  """

  def weighted_loss(y_true, y_pred):
    """
    Calculates the weighted loss for each class and returns the sum.

    Args:
        y_true: True labels (one-hot encoded).
        y_pred: Predicted probabilities.

    Returns:
        The total weighted loss.
    """

    # Initialize loss to zero
    loss = 0.0

    for i in range(len(pos_weights)):
      # For each class, add average weighted loss for that class
      loss += - pos_weights[i] * K.mean(y_true[:, i] * K.log(y_pred[:, i] + epsilon))  \
             - neg_weights[i] * K.mean((1 - y_true[:, i]) * K.log(1 - y_pred[:, i] + epsilon))

    return loss

  return weighted_loss


In [1]:
def get_weighted_loss(pos_weights, neg_weights, epsilon=1e-7):
    """
    Return weighted loss function given negative weights and positive weights.

    Args:
      pos_weights (np.array): array of positive weights for each class, size (num_classes)
      neg_weights (np.array): array of negative weights for each class, size (num_classes)
    
    Returns:
      weighted_loss (function): weighted loss function
    """
    def weighted_loss(y_true, y_pred):
        """
        Return weighted loss value. 

        Args:
            y_true (Tensor): Tensor of true labels, size is (num_examples, num_classes)
            y_pred (Tensor): Tensor of predicted labels, size is (num_examples, num_classes)
        Returns:
            loss (Float): overall scalar loss summed across all classes
        """
        # initialize loss to zero
        loss = 0.0
        
        
        for i in range(len(pos_weights)):
            # for each class, add average weighted loss for that class 
            loss += K.mean(-(pos_weights[i] *y_true[:,i] * K.log(y_pred[:,i] + epsilon) 
                             + neg_weights[i]* (1 - y_true[:,i]) * K.log( 1 - y_pred[:,i] + epsilon))) #complete this line
        return loss
    
        
    return weighted_loss

In [ ]:
import tensorflow as tf  # Assuming TensorFlow is used for the model

def load_model(model_path):
  """Loads a pre-trained model from the specified path.

  Args:
      model_path: Path to the saved model directory.

  Returns:
      The loaded TensorFlow model.
  """

  # Load the model architecture and weights from the specified path
  model = tf.keras.models.load_model(model_path)

  return model

def load_image(path, target_size=(128, 128)):
  """Loads and preprocesses an image for model input.

  Args:
      path: Path to the image file.
      target_size: Target size for resizing the image (optional).

  Returns:
      A preprocessed image tensor ready for model prediction.
  """

  # Load the image using an appropriate library (e.g., OpenCV, Pillow)
  img = cv2.imread(path)  # Example using OpenCV

  # Preprocess the image (resize, normalization, etc.) based on your model's requirements
  img = cv2.resize(img, target_size)
  img = img.astype('float32') / 255.0  # Normalize (assuming 0-255 pixel range)

  # Add a batch dimension for compatibility with most models
  img = np.expand_dims(img, axis=0)

  return img

def make_prediction(model, image_path, class_labels):
  """Performs prediction on an image using the loaded model and returns predicted class(es).

  Args:
      model: The loaded TensorFlow model.
      image_path: Path to the image file.
      class_labels: List of class labels corresponding to model outputs.

  Returns:
      A list of predicted class labels (or the most likely class if the model is single-class).
  """

  # Load the image using the load_image function
  image = load_image(image_path)

  # Make predictions using the model
  predictions = model.predict(image)

  # Process predictions based on model output format:
  if len(predictions.shape) == 2:  # Multi-class model (one-hot encoded predictions)
    predicted_classes = np.argmax(predictions, axis=1)  # Get class indices with highest probabilities
    predicted_labels = [class_labels[i] for i in predicted_classes]  # Convert indices to labels
  else:  # Single-class model (single probability output)
    predicted_class_proba = predictions[0][0]  # Assuming the first element is the probability
    predicted_label = class_labels[0] if predicted_class_proba > 0.5 else 'Negative'  # Example thresholding

  return predicted_labels  # Or predicted_label for single-class model

# Example usage (replace with your actual model path, image path, and class labels)
model = load_model('/content/chest_xray_model_pretrained.keras')
image_path = '/content/WhatsApp Image 2024-03-11 at 12.59.12 PM.jpeg'
vlabels = ['Cardiomegaly', 'Emphysema', 'Effusion', 'Hernia', 'Infiltration', 'Mass', 'Nodule', 'Atelectasis',
           'Pneumothorax', 'Pleural_Thickening', 'Pneumonia', 'Fibrosis', 'Edema', 'Consolidation']

predicted_classes = make_prediction(model, image_path, vlabels)

# Display or use the predicted classes as needed
print("Predicted classes:", predicted_classes)


In [3]:
def weighted_loss(y_true, y_pred):
    # Define the weighted loss function here
    # ...
    def loss(y_true, y_pred):
        # Initialize loss to zero
        loss_value = 0.0

        # Compute weighted binary cross-entropy loss for each class
        for i in range(len(pos_weights)):
            loss_value += - pos_weights[i] * K.mean(y_true[:, i] * K.log(y_pred[:, i] + epsilon)) \
                          - neg_weights[i] * K.mean((1 - y_true[:, i]) * K.log(1 - y_pred[:, i] + epsilon))

        return loss_value
    return loss


In [ ]:
import tensorflow as tf
from keras.preprocessing import image
import numpy as np

# Load the saved Keras model
model = tf.keras.models.load_model('G:/colab_models/chest_xray_model_pretrained.keras', custom_objects={'weighted_loss': weighted_loss})

# Preprocess the image
def preprocess_image(image_path, target_size=(320, 320)):
    img = image.load_img(image_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    processed_img = img_array / 255.0  # Normalize pixel values to [0, 1]
    return processed_img

# Perform multi-class classification
def multi_class_classification(image_path, labels):
    processed_img = preprocess_image(image_path)
    predictions = model.predict(processed_img)[0]
    class_predictions = {label: prediction for label, prediction in zip(labels, predictions)}
    return class_predictions

# Example usage:
image_path = "G:/colab_models/WhatsApp Image 2024-03-11 at 12.59.12 PM.jpeg"
labels = ['Cardiomegaly', 'Emphysema', 'Effusion', 'Hernia', 'Infiltration', 'Mass', 'Nodule', 'Atelectasis',
          'Pneumothorax', 'Pleural_Thickening', 'Pneumonia', 'Fibrosis', 'Edema', 'Consolidation']
predictions = multi_class_classification(image_path, labels)
for label, prediction in predictions.items():
    print(f"{label}: {prediction}")


In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from tensorflow.keras.models import load_model

# Load your trained model
model = load_model("G:/colab_models/chest_xray_model_pretrained.keras")

# List of diseases for binary classification
diseases = ['Cardiomegaly', 'Emphysema', 'Effusion', 'Hernia', 'Infiltration', 'Mass', 'Nodule', 'Atelectasis',
           'Pneumothorax', 'Pleural_Thickening', 'Pneumonia', 'Fibrosis', 'Edema', 'Consolidation']

def preprocess_image(image_path):
    # Load and preprocess the image
    img = Image.open(image_path)
    img = img.resize((320, 320))  # Resize to match input size used during training
    img_array = np.array(img)
    img_array = img_array / 255.0  # Normalize pixel values (if necessary)
    return img_array

def predict_image(image_path):
    # Preprocess the image
    img_array = preprocess_image(image_path)
    # Make prediction
    predictions = model.predict(np.expand_dims(img_array, axis=0))[0]  # Extract predictions for the first (and only) image
    # Threshold predictions (e.g., using 0.5 as threshold)
    binary_predictions = [1 if pred >= 0.5 else 0 for pred in predictions]
    return binary_predictions, img_array

# Example usage
image_path = "G:/colab_models/WhatsApp Image 2024-03-11 at 12.59.12 PM.jpeg"
predictions, processed_image = predict_image(image_path)

# Plot the processed image
plt.imshow(processed_image)
plt.axis('off')
plt.title('Processed Image')
plt.show()

print("Predictions:")
for i, disease in enumerate(diseases):
    if predictions[i] == 1:
        print(f"{disease}: Disease present")
    else:
        print(f"{disease}: Disease absent")


In [ ]:
pip install numpy==1.22.0

In [ ]:
pip install --upgrade tensorflow


In [ ]:
import tensorflow as tf
print(tf.keras.__version__)


In [ ]:
from tensorflow.keras.models import load_model


In [ ]:

import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.saving import register_keras_serializable
from tensorflow.keras import backend as K  # Assuming you're using Keras backend
@register_keras_serializable()
def get_weighted_loss(pos_weights, neg_weights, epsilon=1e-7):
  """
  Defines a custom weighted loss function.

  Args:
      pos_weights: List of weights for positive classes.
      neg_weights: List of weights for negative classes.
      epsilon: Small value to avoid log(0) errors.

  Returns:
      A function that calculates the weighted loss.
  """

  def weighted_loss(y_true, y_pred):
    """
    Calculates the weighted loss for each class and returns the sum.

    Args:
        y_true: True labels (one-hot encoded).
        y_pred: Predicted probabilities.

    Returns:
        The total weighted loss.
    """

    # Initialize loss to zero
    loss = 0.0

    for i in range(len(pos_weights)):
      # For each class, add average weighted loss for that class
      loss += - pos_weights[i] * K.mean(y_true[:, i] * K.log(y_pred[:, i] + epsilon))  \
             - neg_weights[i] * K.mean((1 - y_true[:, i]) * K.log(1 - y_pred[:, i] + epsilon))

    return loss

  return weighted_loss


In [4]:




import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from tensorflow.keras.models import load_model

# Load your trained model
model = load_model("D:/final_saved_models/chest_xray_model_small.h5")

# List of diseases for binary classification
diseases = ['Cardiomegaly', 'Emphysema', 'Effusion', 'Hernia', 'Infiltration', 'Mass', 'Nodule', 'Atelectasis',
           'Pneumothorax', 'Pleural_Thickening', 'Pneumonia', 'Fibrosis', 'Edema', 'Consolidation']

def preprocess_image(image_path):
    # Load and preprocess the image
    img = Image.open(image_path)
    img = img.resize((320, 320))  # Resize to match input size used during training
    img_array = np.array(img)
    img_array = img_array / 255.0  # Normalize pixel values (if necessary)
    return img_array

def predict_image(image_path):
    # Preprocess the image
    img_array = preprocess_image(image_path)
    # Make prediction
    predictions = model.predict(np.expand_dims(img_array, axis=0))[0]  # Extract predictions for the first (and only) image
    # Threshold predictions (e.g., using 0.5 as threshold)
    binary_predictions = [1 if pred >= 0.5 else 0 for pred in predictions]
    return binary_predictions, img_array

# Example usage
image_path = "G:/colab_models/WhatsApp Image 2024-03-11 at 12.59.12 PM.jpeg"
predictions, processed_image = predict_image(image_path)

# Plot the processed image
plt.imshow(processed_image)
plt.axis('off')
plt.title('Processed Image')
plt.show()

print("Predictions:")
for i, disease in enumerate(diseases):
    if predictions[i] == 1:
        print(f"{disease}: Disease present")
    else:
        print(f"{disease}: Disease absent")


ValueError: Unknown loss function: 'weighted_loss'. Please ensure you are using a `keras.utils.custom_object_scope` and that this object is included in the scope. See https://www.tensorflow.org/guide/keras/save_and_serialize#registering_the_custom_object for details.

In [ ]:
import tensorflow as tf

print("TensorFlow Version:", tf.__version__)
try:
  print("Keras Version:", tf.keras.__version__)
except AttributeError:
  print("Keras might be a separate installation. You can check its version using:")
  print("```python")
  print("from keras import __version__")
  print("print(__version__)")
  print("```")


In [ ]:
from keras import __version__  

In [ ]:
print(__version__)

In [6]:
import matplotlib as plt
print("TensorFlow Version:", plt.__version__)

TensorFlow Version: 3.5.1
